## Load in Necessary Packages, etc.

In [74]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sys

sys.path.append('D:\substorm-detection\RNN')
sys.path.append('D:\substorm-detection')
sys.path.append('D:\substorm-detection\\baseline')
sys.path.append('D:\substorm-detection\\data')

import rnn_models
import utils
import utils_linear
import plot_utils
import linear_models

%load_ext autoreload
%autoreload 2

sns.set()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load in Data

In [57]:
folder = 'D:\\substorm-detection\\data'
data_fn = folder + '\\short_data.npz'
data_fn = folder + '\\2classes_data128_withsw.npz'
data = np.load(data_fn)
X = data['X']
y = data['y'][:, None]
SW = data['SW']
strength = data['strength']

print(np.shape(X))
print(np.shape(y))
print(np.shape(SW))
print(np.shape(strength))

In [88]:
batch_size = 32

train_test_split = .2
train_val_split = .2
train, test = utils.split_data([X, SW, y, strength], train_test_split, random=False)
train, val = utils.split_data(train, train_val_split, random=True, batch_size=batch_size)
X_train, sw_train, y_train, strength_train = train
X_val, sw_val, y_val, strength_val = val
X_test, sw_test, y_test, strength_test = test

X_train, X_val, X_test = utils.rnn_format_x([X_train, X_val, X_test])
y_train, y_val, y_test = utils.rnn_format_y([y_train, y_val, y_test])

X_train, X_val, X_test = [X_train, sw_train], [X_val, sw_val], [X_test, sw_test]
y_train, y_val, y_test = [y_train, strength_train], [y_val, strength_val], [y_test, strength_test]

C:\Users\Alan\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Alan\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\Alan\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was 

In [135]:
params = {
    'batch_size': batch_size,
    'rnn_hidden_units': 64,
    'n_stacks': 2,
    'fc_hidden_size': 128,
    'n_classes': 2,
    'epochs': 3,
    'verbose': True,
    'time_output_weight': 1e6,
    'rnn_type': 'GRU',
    'output_type': 'time'
}

In [131]:
hist, model = rnn_models.train_functional_rnn_single(X_train, y_train, X_val, y_val, params)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_104 (InputLayer)          (32, 256, 6)         0                                            
__________________________________________________________________________________________________
input_103 (InputLayer)          (32, 128, 243)       0                                            
__________________________________________________________________________________________________
gru_203 (GRU)                   (32, 256, 64)        13632       input_104[0][0]                  
__________________________________________________________________________________________________
gru_201 (GRU)                   (32, 128, 64)        59136       input_103[0][0]                  
__________________________________________________________________________________________________
gru_204 (G

In [136]:
hist, model = rnn_models.train_functional_rnn_combined(X_train, y_train, X_val, y_val, params)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_112 (InputLayer)          (32, 256, 6)         0                                            
__________________________________________________________________________________________________
input_111 (InputLayer)          (32, 128, 243)       0                                            
__________________________________________________________________________________________________
gru_219 (GRU)                   (32, 256, 64)        13632       input_112[0][0]                  
__________________________________________________________________________________________________
gru_217 (GRU)                   (32, 128, 64)        59136       input_111[0][0]                  
__________________________________________________________________________________________________
gru_220 (G

### Training on only solar wind data

In [85]:
params = {
    'batch_size': batch_size,
    'rnn_hidden_units': 128,
    'n_stacks': 2,
    'fc_hidden_size': 128,
    'n_classes': 2,
    'epochs': 3,
    'verbose': True,
    'time_output_weight': 1,
    'rnn_type': 'GRU',
}

In [86]:
hist, model = rnn_models.train_basic_gru(X_train[1], y_train[0], X_val[1], y_val[0], params)

Train on 15456 samples, validate on 2720 samples
Epoch 1/3
15456/15456 [==============================] - 292s 19ms/step - loss: 0.6141 - acc: 0.6613 - true_positive: 0.6544 - false_positive: 0.3265 - val_loss: 0.5661 - val_acc: 0.7114 - val_true_positive: 0.7150 - val_false_positive: 0.2898
Epoch 2/3
15456/15456 [==============================] - 279s 18ms/step - loss: 0.5750 - acc: 0.7044 - true_positive: 0.6619 - false_positive: 0.2520 - val_loss: 0.5683 - val_acc: 0.7114 - val_true_positive: 0.5880 - val_false_positive: 0.1618
Epoch 3/3
15456/15456 [==============================] - 280s 18ms/step - loss: 0.5730 - acc: 0.7070 - true_positive: 0.6592 - false_positive: 0.2445 - val_loss: 0.5653 - val_acc: 0.7129 - val_true_positive: 0.6960 - val_false_positive: 0.2664


### Training on only magnetometer data

In [87]:
hist, model = rnn_models.train_basic_gru(X_train[0], y_train[0], X_val[0], y_val[0], params)

Train on 15456 samples, validate on 2720 samples
Epoch 1/3
15456/15456 [==============================] - 150s 10ms/step - loss: 0.5788 - acc: 0.7075 - true_positive: 0.6965 - false_positive: 0.2810 - val_loss: 0.5437 - val_acc: 0.7485 - val_true_positive: 0.7647 - val_false_positive: 0.2653
Epoch 2/3
15456/15456 [==============================] - 145s 9ms/step - loss: 0.5354 - acc: 0.7494 - true_positive: 0.7336 - false_positive: 0.2326 - val_loss: 0.5373 - val_acc: 0.7423 - val_true_positive: 0.7411 - val_false_positive: 0.2564
Epoch 3/3
15456/15456 [==============================] - 145s 9ms/step - loss: 0.5074 - acc: 0.7666 - true_positive: 0.7427 - false_positive: 0.2085 - val_loss: 0.5187 - val_acc: 0.7640 - val_true_positive: 0.7769 - val_false_positive: 0.2498
